# 01 - First Exploration

First exploration of the data.

## Preliminaires 

### System 

In [ ]:
cd ../

In [ ]:
pwd

### Imports

In [ ]:
import os, sys, logging

from dataclasses import dataclass

In [ ]:
from IPython.display import display, HTML

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

import missingno as msno

In [ ]:
from gbs.etl.extract import Extract

### Data

In [ ]:
extract = Extract()
extract

In [ ]:
data_dir = os.path.join(
    # os.getcwd(),
    extract.base,
    extract.folder,
    extract.subfolder,
)
data_dir

In [ ]:
# # get data
# if (not os.path.exists(data_dir)) or  (not os.listdir(data_dir)):
#     extract.get_all(clean=True, include_production=True)

In [ ]:
# extract.get_all(clean=True, production=True)

In [ ]:
data_dir = "./data/source/"
os.listdir(data_dir)

In [ ]:
!rm data/source/production.zip

In [ ]:
!tree -L 1 ./

In [ ]:
!tree -L 3 data/

In [ ]:
crops = pd.read_csv(os.path.join(data_dir, "crops.csv"))
crops

In [ ]:
country_specs = pd.read_csv(os.path.join(data_dir, "country_specs.csv"))
country_specs.head()

In [ ]:
_path = "./data/source/production/"

fn_list = [
    os.path.join(_path, f) for f in os.listdir(_path) if f.endswith(".csv")
]
fn_list

In [ ]:
# for fn in fn_list:
#     # read file
#     with open(fn, "r", encoding="latin-1") as f:
#         txt = f.read()

#     # clean file
#     txt.replace(",'", ",")

#     # write file
#     with open(fn, "w", encoding="utf8") as f:
#         f.write(txt)

In [ ]:
_path = "./data/source/production/"

In [ ]:
@dataclass
class Production:
    """Production data for a crop in a country in a year"""

    areacodes = pd.read_csv(os.path.join(_path, "areacodes.csv"))
    data_normalized = pd.read_csv(os.path.join(_path, "data_normalized.csv"))
    flags = pd.read_csv(os.path.join(_path, "flags.csv"))
    itemcodes = pd.read_csv(os.path.join(_path, "itemcodes.csv"))

## Exploration 

### Country Specs 

#### Display

In [ ]:
country_specs.head(10)

In [ ]:
country_specs.tail(10)

In [ ]:
country_specs.sample(10)

#### Structure

In [ ]:
country_specs.shape

In [ ]:
country_specs.columns

In [ ]:
country_specs.dtypes

In [ ]:
country_specs.info()

In [ ]:
crops.dtypes.value_counts()

In [ ]:
for dtype in ["object", "float", "int"]:
    selected_dtype = country_specs.select_dtypes(include=[dtype])
    display(selected_dtype.columns)

In [ ]:
_num = country_specs.select_dtypes(include=["number"])
_num

In [ ]:
_num_cols = _num.columns.tolist()
[i for i in _num_cols if ("code" not in i) and ("id" not in i)]

**Conclusion**

No data in the crop dataset.

#### Nan

In [ ]:
crops.isna().sum()

In [ ]:
tmp = crops.isna().mean().round(2)
tmp[tmp > 0.00]

In [ ]:
len(tmp[tmp > 0.00]) / len(tmp)

In [ ]:
tmp = crops.isna().mean(axis=1).round(2)
tmp.value_counts().sort_index()

In [ ]:
msno.matrix(country_specs)

**Conclusion**

- delete is_EU27 and is_south_america

In [ ]:
# delete is_EU27 and is_south_america

In [ ]:
drop_cols = ["is_EU27", "is_south_america"]
country_specs = country_specs.drop(columns=drop_cols)
msno.matrix(country_specs)

In [ ]:
tmp = country_specs.isna().mean(axis=1).round(2)
tmp.value_counts().sort_index()

In [ ]:
threshold = 0.2
tmp[tmp >= threshold]

In [ ]:
drop_idxs = tmp[tmp > threshold].index
country_specs.loc[drop_idxs, :]

**Conclusion**

- countries with Nan rate > 0.3 are Island or -100k pop (Monaco, Antigua)

In [ ]:
country_specs = country_specs.drop(
    index=drop_idxs, columns=drop_cols, errors="ignore"
)
msno.matrix(country_specs)

In [ ]:
num_cols = country_specs.select_dtypes(
    include=["float", "int"]
).columns.tolist()

num_cols = [i for i in num_cols if "code" not in i]

categ_cols = [
    "alpha_3_code",
    "FAO_country_name",
    "exiobase_region_name",
    "globio_country_code",
    "globio_country_name",
    "USS30_region_name",
]


country_specs = country_specs.loc[:, num_cols + categ_cols]

In [ ]:
country_specs

**Conclusion**

- No relevant data in the crop dataset

#### Data Inspection

In [ ]:
country_specs.FAO_country_name.value_counts()

In [ ]:
country_specs.FAO_country_name.nunique()

In [ ]:
country_specs.USS30_region_name.value_counts()

In [ ]:
country_specs.FAO_country_name.value_counts().value_counts()

In [ ]:
country_specs.groupby("exiobase_region_name").FAO_country_name.count()

#### DataSet Conclusion

- No relevant data in the crop dataset

### Crops 

#### Wheat Selection

In [ ]:
feature = "Wheat"
crops = crops.loc[crops.item_name == feature, :]

In [ ]:
item_cols = [i for i in crops.columns if "item" in i]
item_cols

In [ ]:
crops.drop(columns=item_cols, inplace=True, errors="ignore")
crops

In [ ]:
crops

#### Display

In [ ]:
crops.head(10)

In [ ]:
crops.tail(10)

In [ ]:
crops.sample(10)

#### Structure

In [ ]:
crops.drop(columns="id", inplace=True, errors="ignore")

In [ ]:
crops.shape

In [ ]:
crops.columns

In [ ]:
crops.dtypes

In [ ]:
crops.info()

In [ ]:
crops.dtypes.value_counts()

In [ ]:
for dtype in ["object", "float", "int"]:
    selected_dtype = crops.select_dtypes(include=[dtype])
    display(selected_dtype.columns)

In [ ]:
_num = crops.select_dtypes(include=["number"])
_num

In [ ]:
_feat_cols = [i for i in _num.columns if "msa" in i]
_feat_cols

#### Separation Static/Dynamic

In [ ]:
categ_cols = [i for i in crops.columns if "msa" not in i]
static_cols = [i for i in crops.columns if "static" in i]
dynamic_cols = [i for i in crops.columns if "dynamic" in i]

display(categ_cols)
display(static_cols)
display(dynamic_cols)

In [ ]:
crops_static = crops.loc[:, categ_cols + static_cols]
crops_static.head()

In [ ]:
crops_dynamic = crops.loc[:, categ_cols + dynamic_cols]
crops_dynamic

# BE CAREFULL => in the report we do have to distinguish between terestrial static, dynamic and marine static, dynamic

#### Summize

crops_static

In [ ]:
crops_static

In [ ]:
sum_static = crops_static.iloc[:, 1:].sum(axis=1)
sum_static

In [ ]:
crops_static = crops_static.iloc[:, :1]
crops_static["sum_static"] = sum_static.values

#### Data Inspection

In [ ]:
crops_static.describe()

In [ ]:
crops.globio_country_code.value_counts()

In [ ]:
crops.globio_country_code.nunique()

**Conclusion**

????

#### Merge Country Specs and crops

In [ ]:
country_specs

In [ ]:
# merged = pd.merge(left=_country_specs, right=crops, on="globio_country_code",how="left")
# merged

In [ ]:
tmp = country_specs.loc[:, ["globio_country_code", "globio_country_name"]]
tmp.index = tmp.globio_country_code
tmp.drop(columns="globio_country_code", inplace=True)
tmp.index.name = None
tmp = tmp.to_dict().get("globio_country_name")
tmp

In [ ]:
crops_static["globio_country_name"] = crops_static.globio_country_code.apply(
    lambda i: tmp.get(i, np.nan)
)
crops_static

#### Nan

In [ ]:
crops_static.isna().sum()

**Conclusion**

???

**Conclusion**
???

## Production

#### Table Analysis

In [ ]:
Production.areacodes

In [ ]:
Production.flags

In [ ]:
flags = {
    k: v
    for k, v in zip(
        Production.flags.Flag.values, Production.flags.Description.values
    )
}
flags

In [ ]:
Production.itemcodes

In [ ]:
data = Production.data_normalized
data

In [ ]:
data["Flag_value"] = data.Flag.apply(lambda i: flags[i])
data

In [ ]:
data.columns

#### Feature selection

In [ ]:
cols = [
    # "Area Code",
    # "Area Code (M49)",
    "Area",
    # "Item Code",
    # "Item Code (CPC)",
    "Item",
    # "Element Code",
    "Element",
    # "Year Code",
    "Year",
    "Unit",
    "Value",
    # "Flag",
    # "Note",
    "Flag_value",
]

In [ ]:
data = data.loc[:, cols]
data

In [ ]:
data.Element.nunique()

In [ ]:
data.Element.value_counts()

In [ ]:
data.Item.value_counts()

In [ ]:
data_weat = data.loc[data.Item.str.lower().str.contains("wheat"), :]
data_weat

In [ ]:
data_weat_2019 = data_weat.loc[data_weat.Year == 2019, :]
data_weat_2019

In [ ]:
data_weat_2019_h = data_weat_2019.loc[data_weat_2019.Unit == "ha", :]

In [ ]:
data_weat_2019_h

In [ ]:
data_weat_2019_h.columns

In [ ]:
cols = [
    "Area",
    # "Item",
    # "Element",
    # "Year",
    # "Unit",
    "Value",
    "Flag_value",
]

data_weat_2019_h = data_weat_2019_h.loc[:, cols]
data_weat_2019_h

#### Display 

In [ ]:
data_weat_2019_h.head(10)

In [ ]:
data_weat_2019_h.tail(10)

In [ ]:
data_weat_2019_h.sample(10)

#### Structure

In [ ]:
data_weat_2019_h.shape

In [ ]:
data_weat_2019_h.info()

#### Nan

In [162]:
data_weat_2019_h.isna().sum()

Area          0
Value         0
Flag_value    0
dtype: int64

In [163]:
data_weat_2019_h

,Area,Value,Flag_value
11474,Afghanistan,2334000.0,Official figure
30465,Albania,57330.0,Official figure
51223,Algeria,1974987.0,Official figure
64909,Angola,3357.0,Estimated value
99174,Argentina,6050953.0,Official figure
...,...,...,...
4044324,Small Island Developing States,167.0,Estimated value
4051501,Low Income Food Deficit Countries,32141.0,Estimated value
4082626,Low Income Food Deficit Countries,8732855.0,Official figure
4090319,Net Food Importing Developing Countries,33608.0,Estimated value


data_weat_2019_h.shape

## Final Merge

### Keys Analysis

In [187]:
data_weat_2019_h.sort_values("Area", inplace=True, ascending=True)
data_weat_2019_h.rename(columns={"Value": "km2"}, inplace=True)
data_weat_2019_h

,Area,km2,Flag_value
11474,Afghanistan,2334000.0,Official figure
3180706,Africa,9647204.0,Estimated value
3149126,Africa,21795.0,Estimated value
30465,Albania,57330.0,Official figure
51223,Algeria,1974987.0,Official figure
...,...,...,...
3105324,World,1838123.0,Estimated value
3141552,World,215748027.0,Official figure
3052765,Yemen,57466.0,Official figure
3077044,Zambia,22706.0,Official figure


####### BE CARREFULLL KM2 is supposed

(202, 3)

In [172]:
crops_static.sort_values("globio_country_name", ascending=True, inplace=True)
crops_static

/tmp/ipykernel_152315/114684046.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crops_static.sort_values("globio_country_name", ascending=True, inplace=True)


,globio_country_code,sum_static,globio_country_name
0,40,0.002951,Austria
59,56,0.001293,Belgium
105,100,0.002051,Bulgaria
187,191,0.001951,Croatia
250,196,0.005020,Cyprus
318,203,0.002068,Czech Republic
380,208,0.001343,Denmark
422,233,0.002742,Estonia
455,246,0.006413,Finland
488,250,0.001674,France


In [174]:
country_specs.sort_values("FAO_country_name", ascending=True, inplace=True)
country_specs

,exiobase_region_id,alpha_3_code,FAO_country_name,exiobase_region_name,globio_country_code,globio_country_name,USS30_region_name
0,45.0,AFG,Afghanistan,RoW Asia and Pacific,4,Afghanistan,Rest S Asia
1,47.0,ALB,Albania,RoW Europe,8,Albania,C.Europe
2,48.0,DZA,Algeria,RoW Africa,12,Algeria,N.Africa
3,45.0,ASM,American Samoa,RoW Asia and Pacific,16,American Samoa,Oceania
5,48.0,AGO,Angola,RoW Africa,24,Angola,Rest S Africa
...,...,...,...,...,...,...,...
236,45.0,WLF,Wallis and Futuna Islands,RoW Asia and Pacific,876,Wallis and Futuna,Oceania
237,48.0,ESH,Western Sahara,RoW Africa,732,Western Sahara,N.Africa
238,49.0,YEM,Yemen,RoW Middle East,887,Yemen,M.East
239,48.0,ZMB,Zambia,RoW Africa,894,Zambia,Rest S Africa


In [173]:
crops_static.shape

(27, 3)

In [176]:
data_weat_2019_h.shape

(202, 3)

In [175]:
country_specs.shape

(222, 7)

### data vs country ON FAO_country_name

In [178]:
merge1 = pd.merge(
    left=country_specs,
    right=data_weat_2019_h,
    left_on="FAO_country_name",
    right_on="Area",
    how="outer",
    indicator=True,
)

merge1

,exiobase_region_id,alpha_3_code,FAO_country_name,exiobase_region_name,globio_country_code,globio_country_name,USS30_region_name,Area,Value,Flag_value,_merge
0,45.0,AFG,Afghanistan,RoW Asia and Pacific,4.0,Afghanistan,Rest S Asia,Afghanistan,2334000.0,Official figure,both
1,47.0,ALB,Albania,RoW Europe,8.0,Albania,C.Europe,Albania,57330.0,Official figure,both
2,48.0,DZA,Algeria,RoW Africa,12.0,Algeria,N.Africa,Algeria,1974987.0,Official figure,both
3,45.0,ASM,American Samoa,RoW Asia and Pacific,16.0,American Samoa,Oceania,NaN,NaN,NaN,left_only
4,48.0,AGO,Angola,RoW Africa,24.0,Angola,Rest S Africa,Angola,3357.0,Estimated value,both
...,...,...,...,...,...,...,...,...,...,...,...
302,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Western Asia,10753795.0,Official figure,right_only
303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Western Asia,103.0,Estimated value,right_only
304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Western Europe,9064853.0,Official figure,right_only
305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,World,1838123.0,Estimated value,right_only


In [179]:
merge1.rename(columns={"_merge": "_merge_1"}, inplace=True)
merge1

,exiobase_region_id,alpha_3_code,FAO_country_name,exiobase_region_name,globio_country_code,globio_country_name,USS30_region_name,Area,Value,Flag_value,_merge_1
0,45.0,AFG,Afghanistan,RoW Asia and Pacific,4.0,Afghanistan,Rest S Asia,Afghanistan,2334000.0,Official figure,both
1,47.0,ALB,Albania,RoW Europe,8.0,Albania,C.Europe,Albania,57330.0,Official figure,both
2,48.0,DZA,Algeria,RoW Africa,12.0,Algeria,N.Africa,Algeria,1974987.0,Official figure,both
3,45.0,ASM,American Samoa,RoW Asia and Pacific,16.0,American Samoa,Oceania,NaN,NaN,NaN,left_only
4,48.0,AGO,Angola,RoW Africa,24.0,Angola,Rest S Africa,Angola,3357.0,Estimated value,both
...,...,...,...,...,...,...,...,...,...,...,...
302,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Western Asia,10753795.0,Official figure,right_only
303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Western Asia,103.0,Estimated value,right_only
304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Western Europe,9064853.0,Official figure,right_only
305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,World,1838123.0,Estimated value,right_only


data_weat_2019_h.head()

### Merge1 v crops 

In [180]:
merge2 = pd.merge(
    left=merge1,
    right=crops_static,
    left_on="globio_country_name",
    right_on="globio_country_name",
    how="outer",
    indicator=True,
)

merge2

,exiobase_region_id,alpha_3_code,FAO_country_name,exiobase_region_name,globio_country_code_x,globio_country_name,USS30_region_name,Area,Value,Flag_value,_merge_1,globio_country_code_y,sum_static,_merge
0,45.0,AFG,Afghanistan,RoW Asia and Pacific,4.0,Afghanistan,Rest S Asia,Afghanistan,2334000.0,Official figure,both,NaN,NaN,left_only
1,47.0,ALB,Albania,RoW Europe,8.0,Albania,C.Europe,Albania,57330.0,Official figure,both,NaN,NaN,left_only
2,48.0,DZA,Algeria,RoW Africa,12.0,Algeria,N.Africa,Algeria,1974987.0,Official figure,both,NaN,NaN,left_only
3,45.0,ASM,American Samoa,RoW Asia and Pacific,16.0,American Samoa,Oceania,NaN,NaN,NaN,left_only,NaN,NaN,left_only
4,48.0,AGO,Angola,RoW Africa,24.0,Angola,Rest S Africa,Angola,3357.0,Estimated value,both,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Western Asia,10753795.0,Official figure,right_only,NaN,NaN,left_only
303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Western Asia,103.0,Estimated value,right_only,NaN,NaN,left_only
304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Western Europe,9064853.0,Official figure,right_only,NaN,NaN,left_only
305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,World,1838123.0,Estimated value,right_only,NaN,NaN,left_only


In [181]:
merge2.rename(columns={"_merge": "_merge_2"}, inplace=True)
merge2

,exiobase_region_id,alpha_3_code,FAO_country_name,exiobase_region_name,globio_country_code_x,globio_country_name,USS30_region_name,Area,Value,Flag_value,_merge_1,globio_country_code_y,sum_static,_merge_2
0,45.0,AFG,Afghanistan,RoW Asia and Pacific,4.0,Afghanistan,Rest S Asia,Afghanistan,2334000.0,Official figure,both,NaN,NaN,left_only
1,47.0,ALB,Albania,RoW Europe,8.0,Albania,C.Europe,Albania,57330.0,Official figure,both,NaN,NaN,left_only
2,48.0,DZA,Algeria,RoW Africa,12.0,Algeria,N.Africa,Algeria,1974987.0,Official figure,both,NaN,NaN,left_only
3,45.0,ASM,American Samoa,RoW Asia and Pacific,16.0,American Samoa,Oceania,NaN,NaN,NaN,left_only,NaN,NaN,left_only
4,48.0,AGO,Angola,RoW Africa,24.0,Angola,Rest S Africa,Angola,3357.0,Estimated value,both,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Western Asia,10753795.0,Official figure,right_only,NaN,NaN,left_only
303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Western Asia,103.0,Estimated value,right_only,NaN,NaN,left_only
304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Western Europe,9064853.0,Official figure,right_only,NaN,NaN,left_only
305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,World,1838123.0,Estimated value,right_only,NaN,NaN,left_only


,exiobase_region_id,alpha_3_code,FAO_country_name,exiobase_region_name,globio_country_code_x,globio_country_name,USS30_region_name,Area,Value,Flag_value,_merge_1,globio_country_code_y,sum_static,_merge_2


In [184]:
merge2.loc[merge2.loc[:, "_merge_2"] == "both", :]

,exiobase_region_id,alpha_3_code,FAO_country_name,exiobase_region_name,globio_country_code_x,globio_country_name,USS30_region_name,Area,Value,Flag_value,_merge_1,globio_country_code_y,sum_static,_merge_2
11,1.0,AUT,Austria,Austria,40.0,Austria,W.Europe,Austria,278340.0,Official figure,both,40.0,0.002951,both
19,2.0,BEL,Belgium,Belgium,56.0,Belgium,W.Europe,Belgium,203760.0,Official figure,both,56.0,0.001293,both
33,3.0,BGR,Bulgaria,Bulgaria,100.0,Bulgaria,C.Europe,Bulgaria,1198680.0,Official figure,both,100.0,0.002051,both
55,13.0,HRV,Croatia,Croatia,191.0,Croatia,C.Europe,Croatia,143150.0,Official figure,both,191.0,0.001951,both
57,4.0,CYP,Cyprus,Cyprus,196.0,Cyprus,C.Europe,Cyprus,10590.0,Official figure,both,196.0,0.005020,both
58,5.0,CZE,Czech republic,Czech Republic,203.0,Czech Republic,C.Europe,NaN,NaN,NaN,left_only,203.0,0.002068,both
62,7.0,DNK,Denmark,Denmark,208.0,Denmark,W.Europe,Denmark,573400.0,Official figure,both,208.0,0.001343,both
71,8.0,EST,Estonia,Estonia,233.0,Estonia,C.Europe,Estonia,166980.0,Official figure,both,233.0,0.002742,both
76,10.0,FIN,Finland,Finland,246.0,Finland,W.Europe,Finland,197600.0,Official figure,both,246.0,0.006413,both
77,11.0,FRA,France,France,250.0,France,W.Europe,France,5244250.0,Official figure,both,250.0,0.001674,both


In [185]:
merge2.loc[merge2.loc[:, "_merge_2"] == "both", :]

,exiobase_region_id,alpha_3_code,FAO_country_name,exiobase_region_name,globio_country_code_x,globio_country_name,USS30_region_name,Area,Value,Flag_value,_merge_1,globio_country_code_y,sum_static,_merge_2
11,1.0,AUT,Austria,Austria,40.0,Austria,W.Europe,Austria,278340.0,Official figure,both,40.0,0.002951,both
19,2.0,BEL,Belgium,Belgium,56.0,Belgium,W.Europe,Belgium,203760.0,Official figure,both,56.0,0.001293,both
33,3.0,BGR,Bulgaria,Bulgaria,100.0,Bulgaria,C.Europe,Bulgaria,1198680.0,Official figure,both,100.0,0.002051,both
55,13.0,HRV,Croatia,Croatia,191.0,Croatia,C.Europe,Croatia,143150.0,Official figure,both,191.0,0.001951,both
57,4.0,CYP,Cyprus,Cyprus,196.0,Cyprus,C.Europe,Cyprus,10590.0,Official figure,both,196.0,0.005020,both
58,5.0,CZE,Czech republic,Czech Republic,203.0,Czech Republic,C.Europe,NaN,NaN,NaN,left_only,203.0,0.002068,both
62,7.0,DNK,Denmark,Denmark,208.0,Denmark,W.Europe,Denmark,573400.0,Official figure,both,208.0,0.001343,both
71,8.0,EST,Estonia,Estonia,233.0,Estonia,C.Europe,Estonia,166980.0,Official figure,both,233.0,0.002742,both
76,10.0,FIN,Finland,Finland,246.0,Finland,W.Europe,Finland,197600.0,Official figure,both,246.0,0.006413,both
77,11.0,FRA,France,France,250.0,France,W.Europe,France,5244250.0,Official figure,both,250.0,0.001674,both
